In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
#디렉토리 변경해주기
%cd /content/drive/MyDrive/0_Capstone/재믐 작업실/ESN
import pyESN
from pyESN import ESN
import seaborn as sns
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

/content/drive/.shortcut-targets-by-id/1c-Rc3RkRVrFAInyRTOpRuuiHG2TB4l1w/0_Capstone/재믐 작업실/ESN


# 데이터 불러오기

In [ ]:
# 기존 30% 주식 데이터를 불러오는 함수
def load_data(file_name):
    file_path = "/content/drive/MyDrive/0_Capstone/data/merged_completed/" + file_name


    # 주식 데이터 불러오기
    all_data_orign = pd.read_csv(file_path)

    all_data_orign.rename(columns={
        'TRD_DD': 'Timestamp',
        'TDD_CLSPRC': 'Close',
        'TDD_OPNPRC': 'Open',
        'TDD_HGPRC': 'High',
        'TDD_LWPRC': 'Low',
        'ACC_TRDVOL': 'Volume'
    }, inplace=True)

    all_data_orign['Timestamp'] = pd.to_datetime(all_data_orign['Timestamp'])

    # 쉼표 제거하기
    def remove_commas(value):
        if isinstance(value, str):
            return int(value.replace(',', ''))
        return value

    all_data_orign = all_data_orign.applymap(remove_commas)

    #df 날짜 역순변경
    all_data_orign["TRD_DD2"] = pd.to_datetime(all_data_orign["Timestamp"])
    all_data_orign.sort_values(by =['TRD_DD2'],ascending = True, inplace= True)
    # trend.head()
    all_data_orign.drop(columns = 'TRD_DD2', inplace = True)
    all_data_orign.reset_index(drop = True, inplace = True)

    columns_to_drop = ['Open', 'High', 'Low', 'Volume']
    all_data_orign.drop(columns=columns_to_drop, inplace=True)

    #예측 정확도 계산에 이용될 30% 데이터 추출
    N = len(all_data_orign)
    traintest_cutoff = int(np.ceil(0.7*N))

    all_data_orign_30  = all_data_orign[traintest_cutoff:]

    return all_data_orign_30

# ESN

In [ ]:
#ESN모델 생성 후 예측 값 반환 함수
def create_esn_model(sparsity, spectral_radius, noise, train_X, test_X):
  #ESN 기본 값 설정
  n_reservoir= 1000
  sparsity=sparsity
  rand_seed=23
  spectral_radius = spectral_radius
  noise = noise

  #각 입력벡터(row)가 8개의 열을 가짐 => 입력데이터의 차원은 8
  n_samples, n_features = train_X.shape

  #모델 생성
  esn = ESN(n_inputs = n_features, #입력 차원의 수(=입력 데이터의 열 수)
      n_outputs = 1, #출력 차원의 수
      n_reservoir = n_reservoir, # reservoir 뉴런의 수
      sparsity=sparsity, # recurrent weights 중 0으로 설정되는 비율
      random_state=rand_seed, #난수 생성을 위한 시드값
      spectral_radius = spectral_radius, # recurrent weight matrix의 스펙트럴 반지름(고유값 중 최대 값)
      noise=noise )   #각 뉴런에 추가되는 노이즈(정규화) -> 과적합방지

  #훈련 데이터 설정
  trainlen = len(train_X) #훈련 데이터의 길이
  futureTotal=len(test_X) #예측할 미래 데이터의 전체 수
  pred_tot=np.zeros(futureTotal) #예측결과를 저장할 배열

  # 훈련 데이터로 ESN 모델 훈련
  esn.fit(train_X, train_Y)

  #예측 수행
  pred_tot = esn.predict(test_X)
  pred_tot = pred_tot.flatten()

  #backtesting1, 2에 사용에 필요한 것 반환
  return pred_tot

In [ ]:
#트렌드 데이터 불러오는 함수
def load_trend_data(file_name):
  file_name = file_name.split('.')[0]
  file_path = "/content/drive/MyDrive/0_Capstone/data/merged_trend/" + file_name + "_preprocessed.csv"
  all_data= pd.read_csv(file_path)
  return all_data

#트렌드 데이터 학습-검증 분할 함수
def ts_train_test(file_name):
  all_data = load_trend_data(file_name)
  #독립변수 선택
  features = ['RSI_sig', 'SMA_sig', 'BBND_sig', 'ROC_sig', 'DPO_sig', 'STOCH_sig', 'MACD_sig', 'GDC_sig']
  data = all_data[features].values
  #종속변수 선택
  target = all_data['TREND_new'].values
  #train/test으로 3:7 이용
  N = len(all_data)
  traintest_cutoff = int(np.ceil(0.7*N))
  #학습 데이터 검증데이터 추출
  train_X,train_Y = data[:traintest_cutoff],target[:traintest_cutoff]
  test_X, test_Y  = data[traintest_cutoff:],target[traintest_cutoff:]

  # Convert data types to float64
  train_X = train_X.astype(np.float64)
  train_Y = train_Y.astype(np.float64)

  return train_X, train_Y, test_X, test_Y

#ESN모델 파라미터 최적화 함수(백테스팅 기준: 수익률)
def best_fit_esn_model(train_X, train_Y, test_X, test_Y,all_data_orign_30):

    # 조정할 파라미터들
    sparsity_set =  [0.1, 0.2, 0.3, 0.4]
    radius_set = [0.9, 1.3, 1.7] # 순환 가중치 행렬의 스펙트럴 반지름 설정
    noise_set = [0.001, 0.01, 0.1] # 각 뉴런에 추가되는 노이즈 설정

    # 설정된 값들의 크기 계산
    sparsity_set_size = len(sparsity_set)
    radius_set_size = len(radius_set)
    noise_set_size = len(noise_set)

    # MSE 값 저장할 3D 배열 초기화
    loss = np.zeros([sparsity_set_size, radius_set_size, noise_set_size])

    # sparsity, 반지름, 노이즈 설정 반복
    for s in range(sparsity_set_size):
        sparsity = sparsity_set[s]
        for l in range(radius_set_size):
            rho = radius_set[l]
            for j in range(noise_set_size):
                noise = noise_set[j]

                #해당 파라미터조건으로 모델 생성 및 예측 결과 출력
                pred_tot= create_esn_model(sparsity, rho, noise, train_X, test_X)

                #시그널 생성
                all_data_orign_30 = making_sig(pred_tot, test_Y, all_data_orign_30)

                # 현재 설정에 모델 평가(백테스팅) 및 결과 저장
                loss[s, l, j] =result_profit(all_data_orign_30, 'pred_sig')
    max_profit = np.max(loss)
    max_indices = np.where(loss == max_profit)
    best_sparsity = sparsity_set[max_indices[0][0]]
    best_rho = radius_set[max_indices[1][0]]
    best_noise = noise_set[max_indices[2][0]]

    return max_profit, best_sparsity, best_rho, best_noise

In [ ]:
#ESN모델 파라미터 최적화 함수(백테스팅 기준: 일치율)
def best_fit_esn_model2(train_X, train_Y, test_X, test_Y,all_data_orign_30):

    # 조정할 파라미터들
    sparsity_set =  [0.1, 0.2, 0.3, 0.4]
    radius_set = [0.9, 1.3, 1.7] # 순환 가중치 행렬의 스펙트럴 반지름 설정
    noise_set = [0.001, 0.01, 0.1] # 각 뉴런에 추가되는 노이즈 설정

    # 설정된 값들의 크기 계산
    sparsity_set_size = len(sparsity_set)
    radius_set_size = len(radius_set)
    noise_set_size = len(noise_set)

    # MSE 값 저장할 3D 배열 초기화
    loss = np.zeros([sparsity_set_size, radius_set_size, noise_set_size])

    # sparsity, 반지름, 노이즈 설정 반복
    for s in range(sparsity_set_size):
        sparsity = sparsity_set[s]
        for l in range(radius_set_size):
            rho = radius_set[l]
            for j in range(noise_set_size):
                noise = noise_set[j]

                #해당 파라미터조건으로 모델 생성 및 예측 결과 출력
                pred_tot= create_esn_model(sparsity, rho, noise, train_X, test_X)

                #시그널 생성
                all_data_orign_30 = making_sig(pred_tot, test_Y, all_data_orign_30)

                # 현재 설정에 모델 평가(백테스팅) 및 결과 저장
                loss[s, l, j] =match_backtest(all_data_orign_30) #matching_percentage
    max_match = np.max(loss)
    max_indices = np.where(loss == max_match)
    best_sparsity = sparsity_set[max_indices[0][0]]
    best_rho = radius_set[max_indices[1][0]]
    best_noise = noise_set[max_indices[2][0]]

    return max_match, best_sparsity, best_rho, best_noise

In [ ]:
#ESN모델 파라미터 최적화 함수(백테스팅 기준: 0.8*수익률 + 0.2*일치율)
def best_fit_esn_model3(train_X, train_Y, test_X, test_Y,all_data_orign_30):

    # 조정할 파라미터들
    sparsity_set =  [0.1, 0.2, 0.3, 0.4]
    radius_set = [0.9, 1.3, 1.7] # 순환 가중치 행렬의 스펙트럴 반지름 설정
    noise_set = [0.001, 0.01, 0.1] # 각 뉴런에 추가되는 노이즈 설정

    # 설정된 값들의 크기 계산
    sparsity_set_size = len(sparsity_set)
    radius_set_size = len(radius_set)
    noise_set_size = len(noise_set)

    # MSE 값 저장할 3D 배열 초기화
    loss = np.zeros([sparsity_set_size, radius_set_size, noise_set_size])

    # sparsity, 반지름, 노이즈 설정 반복
    for s in range(sparsity_set_size):
        sparsity = sparsity_set[s]
        for l in range(radius_set_size):
            rho = radius_set[l]
            for j in range(noise_set_size):
                noise = noise_set[j]

                #해당 파라미터조건으로 모델 생성 및 예측 결과 출력
                pred_tot= create_esn_model(sparsity, rho, noise, train_X, test_X)

                #시그널 생성
                all_data_orign_30 = making_sig(pred_tot, test_Y, all_data_orign_30)

                #수익률 생성
                profits = result_profit(all_data_orign_30, 'pred_sig')
                #일치율 생성
                matching_percentage = match_backtest(all_data_orign_30)

                # 현재 설정에 모델 평가(백테스팅) 및 결과 저장
                loss[s, l, j] = 0.8*profits + 0.2*matching_percentage
    max_profit_match = np.max(loss)
    max_indices = np.where(loss == max_profit_match)
    best_sparsity = sparsity_set[max_indices[0][0]]
    best_rho = radius_set[max_indices[1][0]]
    best_noise = noise_set[max_indices[2][0]]

    return max_profit_match, best_sparsity, best_rho, best_noise

In [ ]:
#backtesting1에 사용될 시그널 생성, backtesting2에 사용될 시그널 생성하여 all_data_orign_30에 붙이기
def making_sig(pred_tot, test_Y, all_data_orign_30):
 #일단 예측 데이터 붙여
 all_data_orign_30['pred'] = pred_tot
 #예측값을 시그널로 변경
 # 조건에 따라 pred_sig 열을 추가
 all_data_orign_30['pred_sig'] = all_data_orign_30['pred'].apply(lambda x: 0 if -0.7 < x < 0.7 else (1 if x >= 0.7 else -1))
 all_data_orign_30['pred_sig'] = all_data_orign_30['pred_sig'].astype(int) #시그널만 int형으로 고치기(수익률 계산 함수에서 오류없이 사용하기 위함)

 #인덱스 재편성
 all_data_orign_30.reset_index(drop=True, inplace=True)

 #일단 실제 정답값 붙여
 all_data_orign_30['trend_30'] = test_Y

 # 조건에 따라 pred_sig 열을 추가
 all_data_orign_30['trend_30_sig'] = all_data_orign_30['trend_30'].apply(lambda x: 0 if -0.7 < x < 0.7 else (1 if x >= 0.7 else -1))
 all_data_orign_30['trend_30_sig'] = all_data_orign_30['trend_30_sig'].astype(int) #시그널만 int형으로 고치기(수익률 계산 함수에서 오류없이 사용하기 위함)

 return all_data_orign_30


# BACKTESTING 함수 (최적화용)

In [ ]:
#시그널 수익률 계산 함수
def result_profit(df, sig_column):
  # 초기 자본금 설정
  initial_capital = 1000000000000000  #1000조

  # 보유 주식 수와 자본금 추적
  shares_held = 0
  capital = initial_capital
  capital_history = [capital]

  # 매수, 매도, 또는 보유 결정에 따른 자본금 변화 계산
  for i in range(1, len(df)):
      if df[sig_column][i] == 1:  # Buy 시그널인 경우
          shares_to_buy = capital // df['MKTCAP'][i]  # 보유 가능한 주식 수 계산
          shares_held += shares_to_buy
          capital -= shares_to_buy * df['MKTCAP'][i]

      elif df[sig_column][i] == -1:  # Sell 시그널인 경우
          capital += shares_held * df['MKTCAP'][i]  # 보유 주식 매도
          shares_held = 0

      if df[sig_column][i] == 0:  # 0 시그널인 경우 (보유 유지)
            capital_history.append(capital + shares_held * df['MKTCAP'][i])

      capital_history.append(capital + shares_held * df['MKTCAP'][i])  # 자본금 변화 추적

  # 수익률 계산
  returns = (capital_history[-1] - initial_capital) / initial_capital * 100

  # 시각화
  #plt.plot(capital_history)
  #plt.title("Capital History")
  #plt.xlabel("Time")
  #plt.ylabel("Capital")
  #plt.show()

  # 결과 출력
  #print("초기 자본금:", initial_capital)
  #print("최종 자본금:", capital_history[-1])
  #print("수익률: {:.2f}%".format(returns))

  return returns

In [ ]:
#변경된 모듈화에 맞게 일치율 함수 변경 -> 최적화 함수에 수익률 구하는데 옆에 넣어서 더해
def match_backtest(all_data_orign_30):

  # 두 열을 비교하여 일치하는 횟수를 계산(일치하면 1 아니면 0)
  all_data_orign_30['match_count'] = (all_data_orign_30['pred_sig'] == all_data_orign_30['trend_30_sig']).astype(int)

  # 일치하는 횟수를 백분율로 계산
  matching_percentage = (all_data_orign_30['match_count'].sum() / len(all_data_orign_30)) * 100

  return matching_percentage

# BACKTESTING 함수 (그래프 출력용)

In [ ]:
def result_profit_graph(df, sig_column):
  # 초기 자본금 설정
  initial_capital = 1000000000000000  #1000조

  # 보유 주식 수와 자본금 추적
  shares_held = 0
  capital = initial_capital
  capital_history = [capital]

  # 매수, 매도, 또는 보유 결정에 따른 자본금 변화 계산
  for i in range(1, len(df)):
      if df[sig_column][i] == 1:  # Buy 시그널인 경우
          shares_to_buy = capital // df['MKTCAP'][i]  # 보유 가능한 주식 수 계산
          shares_held += shares_to_buy
          capital -= shares_to_buy * df['MKTCAP'][i]

      elif df[sig_column][i] == -1:  # Sell 시그널인 경우
          capital += shares_held * df['MKTCAP'][i]  # 보유 주식 매도
          shares_held = 0

      if df[sig_column][i] == 0:  # 0 시그널인 경우 (보유 유지)
            capital_history.append(capital + shares_held * df['MKTCAP'][i])

      capital_history.append(capital + shares_held * df['MKTCAP'][i])  # 자본금 변화 추적

  # 수익률 계산
  returns = (capital_history[-1] - initial_capital) / initial_capital * 100

  # 시각화
  plt.plot(capital_history)
  plt.title("Capital History")
  plt.xlabel("Time")
  plt.ylabel("Capital")
  plt.show()

  # 결과 출력
  print("초기 자본금:", initial_capital)
  print("최종 자본금:", capital_history[-1])
  print("수익률: {:.2f}%".format(returns))

  return

In [ ]:
def match_backtest_graph(all_data_orign_30):

  # 두 열을 비교하여 일치하는 횟수를 계산(일치하면 1 아니면 0)
  all_data_orign_30['match_count'] = (all_data_orign_30['pred_sig'] == all_data_orign_30['trend_30_sig']).astype(int)

  # 일치하는 횟수를 백분율로 계산
  matching_percentage = (all_data_orign_30['match_count'].sum() / len(all_data_orign_30)) * 100

  #-----정답과 예측 시그널값들이 어떻게 편성되었는지 막대그래프로 확인-------
  # 0, 1, -1 값의 갯수를 계산
  pred_counts = all_data_orign_30['pred_sig'].value_counts().reindex([1, 0, -1], fill_value=0)
  trend_counts = all_data_orign_30['trend_30_sig'].value_counts().reindex([1, 0, -1], fill_value=0)

  # 막대 그래프로 시각화
  labels = ['1', '0', '-1']
  x = range(len(labels))
  width = 0.35

  fig, ax = plt.subplots()
  rects1 = ax.bar(x, pred_counts, width, label='pred_sig')
  rects2 = ax.bar([i + width for i in x], trend_counts, width, label='trend_30_sig')

  ax.set_xlabel('values')
  ax.set_ylabel('count ')
  ax.set_title('pred_sig vs. trend_30_sig')
  ax.set_xticks([i + width/2 for i in x])
  ax.set_xticklabels(labels)
  ax.legend()

  plt.show()

  #-----정답과 예측 시그널값 일치율 원그래프로 확인-----------------
  # 두 열을 비교하여 일치하는 횟수를 계산
  all_data_orign_30['match_count'] = (all_data_orign_30['pred_sig'] == all_data_orign_30['trend_30_sig']).astype(int)

  # 일치하지 않는 횟수 계산
  non_matching_count = len(all_data_orign_30) - all_data_orign_30['match_count'].sum()

  # 일치하는 횟수와 일치하지 않는 횟수를 시각화
  labels = ['match', 'mismatch']
  sizes = [all_data_orign_30['match_count'].sum(), non_matching_count]
  colors = ['lightblue', 'lightcoral']
  explode = (0.1, 0)  # 조각 분리

  plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
  plt.axis('equal')
  plt.title('percentage of matching two columns')
  plt.show()

  return

In [ ]:
def predic_graph(test_Y, pred_tot):
  # 시각화: 실제 값과 예측 값 그래프
  plt.figure(figsize=(12, 6))
  plt.plot(test_Y, label="Actual", color="blue")  # 실제 값 선 색상: 파란색
  plt.plot(pred_tot, label="Predicted", color="red")  # 예측 값 선 색상: 빨간색
  plt.xlabel("Time Step")
  plt.ylabel("Value")
  plt.title("Actual vs. Predicted")
  plt.legend()
  plt.show()

  return

#종목별 최적 모델 생성 후 수익률/ 일치율 그래프 확인

In [ ]:
file_path = "/content/drive/MyDrive/0_Capstone/재믐 작업실/ESN/NEW_TREND_ESN수익률일치율_최적화.csv"


# 최적 파라미터 csv -> df로 불러오기
optimal_para = pd.read_csv(file_path)
optimal_para

,file_name,max_profit_match,best_sparsity,best_rho,best_noise
0,new_KR7005930003.csv,174.182409,0.1,1.7,0.010
1,new_KR7000250001.csv,578.663007,0.4,1.7,0.100
2,new_KR7036570000.csv,97.039102,0.4,1.3,0.001
3,new_KR7051910008.csv,406.648294,0.4,1.3,0.100
4,new_KR7066700006.csv,308.163207,0.2,1.7,0.010
5,new_KR7066970005.csv,1339.837852,0.4,1.3,0.010
6,new_KR7068760008.csv,402.916406,0.1,1.7,0.010
7,new_KR7078600004.csv,562.989906,0.4,1.7,0.010
8,new_KR7096770003.csv,73.047222,0.3,1.3,0.100
9,new_KR7128940004.csv,168.338268,0.2,1.3,0.010


In [ ]:
# 빈 DataFrame 생성
result_df = pd.DataFrame(columns=['file_name', 'best_sparsity', 'best_rho', 'best_noise', 'profits', 'matching_percentage'])

for index, row in optimal_para.iterrows():
    # 파일명 그리고 최적 파라미터 받아와서 변수에 저장
    file_name = row['file_name']
    best_sparsity = row['best_sparsity']
    best_rho = row['best_rho']
    best_noise = row['best_noise']

    all_data_orign_30 = load_data(file_name)  # 기존 데이터 30% 로드해오기
    train_X, train_Y, test_X, test_Y = ts_train_test(file_name)  # 트렌드 데이터 불러오기 및 학습-테스트 데이터 나누기
    pred_tot = create_esn_model(best_sparsity, best_rho, best_noise, train_X, test_X)  # 최적 파라미터로 모델 학습 및 예측결과 저장
    all_data_orign_30 = making_sig(pred_tot, test_Y, all_data_orign_30)  # 시그널 생성하기
    profits = result_profit(all_data_orign_30, 'pred_sig')
    matching_percentage = match_backtest(all_data_orign_30)

    # 결과를 DataFrame에 추가
    result_df = result_df.append({
        'file_name': file_name,
        'best_sparsity': best_sparsity,
        'best_rho': best_rho,
        'best_noise': best_noise,
        'profits': profits,
        'matching_percentage': matching_percentage
    }, ignore_index=True)

# 최종 결과 DataFrame 출력
result_df

,file_name,best_sparsity,best_rho,best_noise,profits,matching_percentage
0,new_KR7005930003.csv,0.1,1.7,0.010,202.001637,62.905501
1,new_KR7000250001.csv,0.4,1.7,0.100,706.657117,66.686567
2,new_KR7036570000.csv,0.4,1.3,0.001,109.123937,48.699764
3,new_KR7051910008.csv,0.4,1.3,0.100,492.117707,64.770642
4,new_KR7066700006.csv,0.2,1.7,0.010,369.809833,61.576705
5,new_KR7066970005.csv,0.4,1.3,0.010,1659.393609,61.614825
6,new_KR7068760008.csv,0.1,1.7,0.010,491.243164,49.609375
7,new_KR7078600004.csv,0.4,1.7,0.010,687.394044,65.373353
8,new_KR7096770003.csv,0.3,1.3,0.100,76.364535,59.777968
9,new_KR7128940004.csv,0.2,1.3,0.010,195.835097,58.350951
